In [ ]:
import pandas as pd
import numpy as np

from IPython.display import display
pd.set_option('display.max_columns', None)

import site
site.addsitedir('/Applications/sam-sdk-2015-6-30-r3/languages/python/')
import PySAM.PySSC as pssc
from urllib.error import HTTPError

In [ ]:
from ipywidgets import interact, Dropdown, Layout

inout_area = {'United States': pd.read_json('Areas_avg_latLong.json').state.tolist()[:-4],'International': pd.read_json('Areas_avg_latLong.json').state.tolist()[-4:]}
area_year = {'Philippines' : ['2017'], 'Vietnam' : ['2016'], 'India' : [str(i) for i in range(2000, 2015)], 'Puerto Rico' : [str(i) for i in range(1998, 2018)]}
year_interval = {'Philippines' : ['60'], 'Vietnam' : ['60'], 'India' : ['60'], 'Puerto Rico' : ['5', '30', '60']}

from itertools import product

state_info = pd.read_json('Areas_avg_latLong.json')
statename = state_info.state.tolist()[:-4]
yearrange = [str(i) for i in range(1998, 2023)]

a = [i for i in product(statename, yearrange)]

def Convert(tup, di):
    for a, b in tup:
        di.setdefault(a, []).append(b)
    return di

dictionary = {}
b = Convert(a, dictionary)

def Merge(dict1, dict2):
    res = {**dict1, **dict2}
    return res
dict3 = Merge(area_year, b)


style = {'description_width': '200px'}
layout = Layout(width='325px')

inout_choice = Dropdown(options = ['United States', 'International'], description = 'In/Out:', style=style, layout=layout)

area1 = Dropdown(description = 'Area:', style=style, layout=layout)

startyear = Dropdown(description = 'Start Year:', style=style, layout=layout)

endyear = Dropdown(description = 'End Year:', style=style, layout=layout)

interval1 = Dropdown(description = 'Temporal Resolution (in Minutes):', style=style, layout=layout)

capacity1 = Dropdown(description = 'Your System Capacity (in MW):', style=style, layout=layout)


In [ ]:
def solar_power_simulation():
    
    global final_data
    appended_data = []
    outsideUS = ['Philippines', 'India', 'Vietnam', 'Puerto Rico']
    # data generation
    try:
        for year in range(int(startyear.value), endyear.value+1):
            
            lat = state_info[state_info['state'] == area1.value]['latitude'].values[0]
            lon = state_info[state_info['state'] == area1.value]['longitude'].values[0]
            api_key = '2aD0f1cpYYogKvIhgzCCEsuBHnVvfGhcaItjJnAU'
            attributes = 'ghi,dhi,dni,wind_speed,air_temperature,solar_zenith_angle'
            year = year
            interval = str(interval1.value)
            leap_year = 'false'
            utc = 'false'
            your_name = 'Justin+Lin'
            reason_for_use = 'beta+testing'
            your_affiliation = 'HTF'
            your_email = 'slin@wvhtf.org'
            mailing_list = 'false'

            if area1.value == 'Philippines':
                dataset = 'philippines-download'
                attributes = 'ghi,dhi,dni,wind_speed,air_temperature'
            if area1.value == 'India':
                dataset = 'suny-india-download'
                attributes = 'ghi,dhi,dni,wspd,surface_temperature,solar_zenith_angle'
            if area1.value == 'Vietnam':
                dataset = 'vietnam-download'
                attributes = 'ghi,dhi,dni,wind_speed,air_temperature'
            if area1.value == 'Puerto Rico':
                dataset = 'puerto-rico-download'
            if area1.value not in outsideUS:
                if (interval1.value == '5') or (interval1.value == '15'):
                    dataset = 'psm3-5min-download'
                else:
                    dataset = 'psm3-2-2-download'
            
        
            url = 'https://developer.nrel.gov/api/nsrdb/v2/solar/{dataset}.csv?wkt=POINT({lon}%20{lat})&names={year}&leap_day={leap}&interval={interval}&utc={utc}&full_name={name}&email={email}&affiliation={affiliation}&mailing_list={mailing_list}&reason={reason}&api_key={api}&attributes={attr}'\
            .format(year=year, dataset=dataset, lat=lat, lon=lon, leap=leap_year, interval=interval, utc=utc, name=your_name, email=your_email, mailing_list=mailing_list, affiliation=your_affiliation, reason=reason_for_use, api=api_key, attr=attributes)
            
            info = pd.read_csv(url, nrows=1)
            timezone, elevation = info['Local Time Zone'], info['Elevation']
        
            df = pd.read_csv(url, skiprows=2)
            df = df.set_index(pd.date_range('1/1/{yr}'.format(yr=year), freq=interval+'Min', periods=525600/int(interval)))

            # SAM Model for solar simulation
            ssc = pssc.PySSC()
            
            # Resource inputs for SAM model:
            # Must be byte strings
            wfd = ssc.data_create()
            ssc.data_set_number(wfd, b'lat', lat)
            ssc.data_set_number(wfd, b'lon', lon)
            ssc.data_set_number(wfd, b'tz', timezone)
            ssc.data_set_number(wfd, b'elev', elevation)
            ssc.data_set_array(wfd, b'year', df.index.year)
            ssc.data_set_array(wfd, b'month', df.index.month)
            ssc.data_set_array(wfd, b'day', df.index.day)
            ssc.data_set_array(wfd, b'hour', df.index.hour)
            ssc.data_set_array(wfd, b'minute', df.index.minute)
            ssc.data_set_array(wfd, b'dn', df['DNI'])
            ssc.data_set_array(wfd, b'df', df['DHI'])
            ssc.data_set_array(wfd, b'wspd', df['Wind Speed'])
            if area1.value == 'Puerto Rico':
                ssc.data_set_array(wfd, b'tdry', df['Air Temperature'])
            else:
                ssc.data_set_array(wfd, b'tdry', df['Temperature'])
        
            # Create SAM compliant object  
            dat = ssc.data_create()
            ssc.data_set_table(dat, b'solar_resource_data', wfd)
            ssc.data_free(wfd)
        
            # Specify the system Configuration
            # Set system capacity in MW
            ssc.data_set_number(dat, b'system_capacity', capacity1.value)
            # Set DC/AC ratio (or power ratio). See https://sam.nrel.gov/sites/default/files/content/virtual_conf_july_2013/07-sam-virtual-conference-2013-woodcock.pdf
            ssc.data_set_number(dat, b'dc_ac_ratio', 1.1)
            # Set tilt of system in degrees
            ssc.data_set_number(dat, b'tilt', 25)
            # Set azimuth angle (in degrees) from north (0 degrees)
            ssc.data_set_number(dat, b'azimuth', 180)
            # Set the inverter efficency
            ssc.data_set_number(dat, b'inv_eff', 96)
            # Set the system losses, in percent
            ssc.data_set_number(dat, b'losses', 14.0757)
            # Specify fixed tilt system (0=Fixed, 1=Fixed Roof, 2=1 Axis Tracker, 3=Backtracted, 4=2 Axis Tracker)
            ssc.data_set_number(dat, b'array_type', 0)
            # Set ground coverage ratio
            ssc.data_set_number(dat, b'gcr', 0.4)
            # Set constant loss adjustment
            ssc.data_set_number(dat, b'adjust:constant', 0)
        
            # execute and put generation results back into dataframe
            mod = ssc.module_create(b'pvwattsv5')
            ssc.module_exec(mod, dat)
            df[b'generation'] = np.array(ssc.data_get_array(dat, b'gen'))
            
            # free the memory
            ssc.data_free(dat)
            ssc.module_free(mod)
            
            appended_data.append(df)
        
        final_data = pd.concat(appended_data)
        
        print(f'\033[1mThis dataset has {final_data.shape[0]} rows and {final_data.shape[1]} columns\033[0m')
        return final_data.head(20)
    
    except HTTPError as err:
        if err.code == 400:
            print("The National Solar Radiance Database doesn't cover your input request")

    

In [ ]:
from matplotlib import pyplot as plt
def daily_plot(df):
    # Using a style
    plt.style.use('Solarize_Light2')
    fig = plt.figure()
    ax = fig.add_subplot(111)
    ax2 = ax.twinx()
    if (area1.value == 'Vietnam') or (area1.value == 'Philippines'):
        df = df.groupby('Hour').sum()[['GHI', 'DNI', 'DHI', b'generation']]
        df[['GHI', 'DNI', 'DHI']].plot(ax=ax, figsize=(15,8), style={'DNI': 'c-o', 'DHI': '-o', 'GHI': 'r-o'}, legend=False)
    else:
        df = df.groupby('Hour').sum()[['GHI', 'DNI', 'DHI', 'Solar Zenith Angle', b'generation']]
        df[['GHI', 'DNI', 'DHI', 'Solar Zenith Angle']].plot(ax=ax, figsize=(15,8), style={'Solar Zenith Angle': '-o', 'DNI': 'c-o', 'DHI': '-o', 'GHI': 'r-o'}, legend=False)
    df[b'generation'].plot(ax=ax2, style={b'generation': '-o'})
    ax.set_ylabel('W/m2')
    ax2.set_ylabel('kW')
    ax.grid()
    ax.legend(loc=2, ncol=5, frameon=False)
    ax2.legend(loc=1, frameon=False)
    ax.set_xticks([i for i in range(1, 25, 2)])
    if startyear.value != endyear.value:
        plt.title(f'Aggregated Daily solar simulation results with {interval1.value} min temporal resolution in {area1.value} from {startyear.value} to {endyear.value}')
    else:
        plt.title(f'Aggregated Daily solar simulation results with {interval1.value} min temporal resolution in {area1.value} in {startyear.value}')


In [ ]:
def monthly_plot(df):
    # Using a style
    plt.style.use('Solarize_Light2')
    fig = plt.figure()
    ax = fig.add_subplot(111)
    ax2 = ax.twinx()
    df = df.groupby('Month').sum()[['GHI', b'generation']]
    df['GHI'].plot(ax=ax, figsize=(15,8), color = 'tab:red', style={'GHI': '-o'}, legend=False)
    df[b'generation'].plot(ax=ax2, style={b'generation': '-o'})
    ax.set_ylabel('W/m2')
    ax2.set_ylabel('kW')
    ax.grid()
    ax.legend(loc=2, ncol=5, frameon=False)
    ax2.legend(loc=1, frameon=False)
    ax.set_xticks(df.index)
    if startyear.value != endyear.value:
        plt.title(f'Aggregated Monthly solar simulation results with {interval1.value} min temporal resolution in {area1.value} from {startyear.value} to {endyear.value}')
    else:
        plt.title(f'Aggregated Monthly solar simulation results with {interval1.value} min temporal resolution in {area1.value} in {startyear.value}')

In [ ]:
def yearly_plot(df):
    # Using a style
    plt.style.use('Solarize_Light2')
    fig = plt.figure()
    ax = fig.add_subplot(111)
    ax2 = ax.twinx()
    df = df.groupby('Year').sum()[['GHI', b'generation']]
    df['GHI'].plot(ax=ax, figsize=(15,8), color = 'tab:red', style = {'GHI' : '-o'}, legend=False)
    df[b'generation'].plot(ax=ax2, style={b'generation': '-o'})
    ax.grid()
    ax.set_ylabel('W/m2')
    ax2.set_ylabel('kW')
    ax.legend(loc=2, ncol=5, frameon=False)
    ax2.legend(loc=1, frameon=False)
    ax.set_xticks(df.index)
    if startyear.value != endyear.value:
        plt.title(f'Aggregated Yearly solar simulation results with {interval1.value} min temporal resolution in {area1.value} from {startyear.value} to {endyear.value}')
    else:
        plt.title(f'Aggregated Yearly solar simulation results with {interval1.value} min temporal resolution in {area1.value} in {startyear.value}')

In [ ]:
@interact(inout = inout_choice, area = area1, start = startyear, end = endyear, interval = interval1, capacity = capacity1)
def print_option(inout, area, start, end, interval, capacity):
    try:
        area1.options = inout_area[inout]
        startyear.options = dict3[area1.value]
        endyear.options = [i for i in range(int(startyear.value), int(dict3[area1.value][-1]) + 1)]
        if (startyear.value >= '2018') and (inout == 'United States'):
            resolution = ['5', '15', '30', '60']
        elif inout == 'United States':
            resolution = ['30', '60']
        else:
            resolution = year_interval[area1.value]
        interval1.options = resolution
        capacity1.options = [i / 10 for i in range(1, 101)]
        
        print(inout, area, start, end, interval, capacity)
    
    except KeyError:
        print('Please choose an area')
    except TypeError: 
        print('Please choose an start year')

In [ ]:
solar_power_simulation()

In [ ]:
daily_plot(final_data)

In [ ]:
monthly_plot(final_data)

In [ ]:
yearly_plot(final_data)